In [156]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error
import pickle

In [157]:
mobile_dataset=pd.read_csv('train.csv')
testing_dataset=pd.read_csv("test.csv")


In [158]:
mobile_dataset.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [159]:
mobile_dataset.price_range.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [160]:
testing_dataset.isna().sum()

id               0
battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
dtype: int64

In [161]:
mobile_dataset.isna().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [162]:
mobile_dataset.duplicated().sum()

0

In [163]:
mobile_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [164]:

import numpy as np


mobile_dataset['sc_size'] = np.sqrt((mobile_dataset['sc_h']**2) + (mobile_dataset['sc_w']**2))
mobile_dataset['sc_size'] = round(mobile_dataset['sc_size']/2.54, 2)


In [165]:
testing_dataset['sc_size'] = np.sqrt((testing_dataset['sc_h']**2) + (testing_dataset['sc_w']**2))
testing_dataset['sc_size'] = round(testing_dataset['sc_size']/2.54, 2)

In [166]:
mobile_dataset.drop(['sc_h', 'sc_w'], axis = 1, inplace = True)

In [167]:
mobile_dataset['pixels'] = mobile_dataset['px_height']*mobile_dataset['px_width']

In [168]:
testing_dataset['pixels'] =testing_dataset['px_height']*testing_dataset['px_width']

In [169]:
mobile_dataset.drop(['px_height', 'px_width'], axis = 1, inplace = True)

In [170]:
mobile_dataset.price_range.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [171]:
X = mobile_dataset.drop(columns=['price_range'])  
y = mobile_dataset['price_range'] 



In [172]:
print(X.head())

   battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  m_dep  \
0            842     0          2.2         0   1       0           7    0.6   
1           1021     1          0.5         1   0       1          53    0.7   
2            563     1          0.5         1   2       1          41    0.9   
3            615     1          2.5         0   0       0          10    0.8   
4           1821     1          1.2         0  13       1          44    0.6   

   mobile_wt  n_cores  pc   ram  talk_time  three_g  touch_screen  wifi  \
0        188        2   2  2549         19        0             0     1   
1        136        3   6  2631          7        1             1     0   
2        145        5   6  2603          9        1             1     0   
3        131        6   9  2769         11        1             0     0   
4        141        2  14  1411         15        1             1     0   

   sc_size   pixels  
0     4.49    15120  
1     6.80  1799140  
2 

In [173]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)



In [174]:
X_test

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,ram,talk_time,three_g,touch_screen,wifi,sc_size,pixels
501,1773,1,2.5,0,16,1,32,0.6,170,6,18,3566,13,1,0,1,7.10,1788480
1924,1982,1,1.6,0,2,1,12,0.2,80,5,20,1824,14,1,1,1,5.25,170191
1710,548,0,1.0,1,0,1,56,0.4,146,4,14,562,8,1,1,1,8.67,1088649
547,1117,1,0.6,1,0,0,31,0.2,146,2,7,2003,14,0,1,1,4.19,2046766
496,1554,1,0.5,1,11,0,5,1.0,124,3,12,3635,18,0,0,0,3.17,623705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204,1524,0,2.5,1,11,1,41,1.0,96,4,20,1936,9,1,1,1,2.76,136160
1937,1396,1,0.6,0,0,0,37,0.7,134,4,15,2694,19,0,0,1,7.18,659120
412,1478,1,0.8,1,12,0,48,0.7,99,5,17,1141,6,0,0,1,2.12,612108
243,1918,0,1.9,0,2,0,10,0.6,110,5,10,1717,16,1,1,1,2.78,1805310


In [232]:
selector = SelectKBest(score_func=chi2, k=11)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)


In [233]:
selected_indices = selector.get_support(indices=True)
selected_feature_names = X.columns[selected_indices]

In [234]:
selected_feature_names

Index(['battery_power', 'fc', 'int_memory', 'mobile_wt', 'n_cores', 'pc',
       'ram', 'talk_time', 'touch_screen', 'sc_size', 'pixels'],
      dtype='object')

In [280]:
testing_dataset1 = testing_dataset[['battery_power', 'fc', 'int_memory', 'mobile_wt', 'pixels',
                                    'sc_size', 'ram', 'pc', 'n_cores', 'talk_time','touch_screen']]

In [281]:
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

In [282]:
scaler = MinMaxScaler()


In [283]:
X_train_scaled = scaler.fit_transform(X_train_selected)


In [284]:
X_test_scaled = scaler.transform(X_test_selected)

In [285]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)


In [286]:
model_rf = RandomForestClassifier(random_state=42)
model_rf.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

In [287]:
y_pred_rf = model_rf.predict(X_test_scaled)


In [288]:
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf}")

Random Forest Accuracy: 0.8925


In [289]:
X_test_new_scaled = scaler.transform(testing_dataset1)
y_pred_rf_test = model_rf.predict(X_test_new_scaled)

/Users/sahilsingh/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


In [290]:
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"Random Forest MSE: {mse_rf}")



Random Forest MSE: 0.1075


In [291]:
with open('model_rf.pkl', 'wb') as file:
    pickle.dump(model_rf, file)


In [292]:
model_lr = LogisticRegression(max_iter=1000)
model_lr.fit(X_train_scaled, y_train)


LogisticRegression(max_iter=1000)

In [293]:
y_pred_lr = model_lr.predict(X_test_scaled)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"Logistic Regression Accuracy: {accuracy_lr}")


Logistic Regression Accuracy: 0.92


In [294]:

mse_lr = mean_squared_error(y_test, y_pred_lr)
print(f"Logistic Regression MSE: {mse_lr}")

Logistic Regression MSE: 0.08


In [295]:
with open('model_lr.pkl', 'wb') as file:
    pickle.dump(model_lr, file)

In [296]:
model_svc = SVC()
model_svc.fit(X_train_scaled, y_train)


SVC()

In [297]:
y_pred_svc = model_svc.predict(X_test_scaled)
accuracy_svc = accuracy_score(y_test, y_pred_svc)
print(f"Support Vector Machine Accuracy: {accuracy_svc}")

Support Vector Machine Accuracy: 0.895


In [298]:
mse_svc = mean_squared_error(y_test, y_pred_svc)
print(f"Support Vector Machine MSE: {mse_svc}")

Support Vector Machine MSE: 0.105


In [299]:
with open('model_svc.pkl', 'wb') as file:
    pickle.dump(model_svc, file)

In [300]:
model_dt = DecisionTreeClassifier()
model_dt.fit(X_train_scaled, y_train)



DecisionTreeClassifier()

In [301]:
y_pred_dt = model_dt.predict(X_test_scaled)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f"Decision Tree Accuracy: {accuracy_dt}")

Decision Tree Accuracy: 0.8675


In [302]:
mse_dt = mean_squared_error(y_test, y_pred_dt)

print(f"Decision Tree MSE: {mse_dt}")

Decision Tree MSE: 0.1325


In [303]:
with open('model_dt.pkl', 'wb') as file:
    pickle.dump(model_dt, file)

In [304]:
model_gb = GradientBoostingClassifier()
model_gb.fit(X_train_scaled, y_train)


GradientBoostingClassifier()

In [305]:
y_pred_gb = model_gb.predict(X_test_scaled)
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print(f"Gradient Boosting Accuracy: {accuracy_gb}")

Gradient Boosting Accuracy: 0.91


In [306]:
mse_gb = mean_squared_error(y_test, y_pred_gb)

print(f"Gradient Boosting MSE: {mse_gb}")

Gradient Boosting MSE: 0.09


In [307]:


with open('model_gb.pkl', 'wb') as file:
    pickle.dump(model_gb, file)

In [308]:
model_nb = GaussianNB()
model_nb.fit(X_train_scaled, y_train)


GaussianNB()

In [309]:
y_pred_nb = model_nb.predict(X_test_scaled)
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f"Gaussian Naive Bayes Accuracy: {accuracy_nb}")

Gaussian Naive Bayes Accuracy: 0.8025


In [310]:
mse_nb = mean_squared_error(y_test, y_pred_nb)
print(f"Gaussian Naive Bayes MSE: {mse_nb}")

Gaussian Naive Bayes MSE: 0.1975


In [311]:
with open('model_nb.pkl', 'wb') as file:
    pickle.dump(model_nb, file)

In [312]:
mse_data = {
    'Model': ['Random Forest', 'Logistic Regression', 'Support Vector Machine', 
              'Decision Tree', 'Gradient Boosting', 'Gaussian Naive Bayes'],
    'MSE': [mse_rf, mse_lr, mse_svc, mse_dt, mse_gb, mse_nb]
}

mse_df = pd.DataFrame(mse_data)
print(mse_df)

                    Model     MSE
0           Random Forest  0.1075
1     Logistic Regression  0.0800
2  Support Vector Machine  0.1050
3           Decision Tree  0.1325
4       Gradient Boosting  0.0900
5    Gaussian Naive Bayes  0.1975


In [313]:

accuracy_data = {
    'Model': ['Random Forest', 'Logistic Regression', 'Support Vector Machine', 
              'Decision Tree', 'Gradient Boosting', 'Gaussian Naive Bayes'],
    'Accuracy': [accuracy_rf, accuracy_lr, accuracy_svc, accuracy_dt, accuracy_gb, accuracy_nb]
}

accuracy_df = pd.DataFrame(accuracy_data)


print(accuracy_df)


                    Model  Accuracy
0           Random Forest    0.8925
1     Logistic Regression    0.9200
2  Support Vector Machine    0.8950
3           Decision Tree    0.8675
4       Gradient Boosting    0.9100
5    Gaussian Naive Bayes    0.8025


In [314]:

mse_df_sorted = mse_df.sort_values(by='MSE')
best_model = mse_df_sorted.iloc[0]['Model']

print(f"The best model based on MSE is: {best_model}")


The best model based on MSE is: Logistic Regression
